In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import datetime
import findLLJ
import matplotlib.pyplot as plt

Additional processing of the surface analysis archive is done separately in the python script and notebook present in the folder `surface_analysis`.

# NYSERDA Data
Downloaded from https://oswbuoysny.resourcepanorama.dnv.com/. CSV files are NOT included in this repository and should be downloaded and saved by the user.

In [2]:
# Replace with appropriate data paths

Buoy = "E06"

if Buoy == "E05":
    lidar = pd.read_csv('/Users/emilydejong/OneDrive - California Institute of Technology/2023NREL/NYSERDA_data/E05_Hudson_North_10_min_avg_20190812_20210919.csv')
    hourly = pd.read_csv('/Users/emilydejong/OneDrive - California Institute of Technology/2023NREL/NYSERDA_data/E05_Hudson_North_hourly_avg_20190812_20210919.csv')
elif Buoy == "E06":
    lidar = pd.read_csv('/Users/emilydejong/OneDrive - California Institute of Technology/2023NREL/NYSERDA_data/E06_Hudson_South_10_min_avg_20190904_20220327.csv')
    hourly = pd.read_csv('/Users/emilydejong/OneDrive - California Institute of Technology/2023NREL/NYSERDA_data/E06_Hudson_South_hourly_avg_20190904_20220327.csv')

/var/folders/_6/ychcj4sn5k7cygz_qf48c5680000gn/T/ipykernel_25198/3876541260.py:9: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,12,13,14,15,16,17,18,19,20,22,23,24,25,26,27,28,29,30,32,33,34,35,36,37,38,39,40,42,43,44,45,46,47,48,49,50,52,53,54,55,56,57,58,59,60,62,63,64,65,66,67,68,69,70,72,73,74,75,76,77,78,79,80,82,83,84,85,86,87,88,89,90,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118) have mixed types. Specify dtype option on import or set low_memory=False.
  lidar = pd.read_csv('/Users/emilydejong/OneDrive - California Institute of Technology/2023NREL/NYSERDA_data/E06_Hudson_South_10_min_avg_20190904_20220327.csv')


In [3]:
lidar = lidar[0:-5]

### Create netcdf file with raw lidar information

In [ ]:
waterT = pd.DataFrame(pd.to_numeric(hourly[' ADCP_ADCPtemp'], errors='coerce'))
waterT['Datetime'] = pd.to_datetime(hourly['timestamp'], errors='coerce')
waterT = waterT.set_index('Datetime')
upsampledT = waterT.resample('10min')
interpolatedT = upsampledT.interpolate(method='linear')

In [ ]:
lidar['Datetime'] = pd.to_datetime(lidar['timestamp'], errors='coerce')
lidar = lidar.set_index('Datetime')

In [ ]:
lidarWaterT = interpolatedT.loc[lidar.index]

In [ ]:
alts = ['18m','38m','58m','78m','98m','118m','138m','158m','178m','198m']
lidar_ws_keys = [] #['timestamp']
lidar_wd_keys = [] #['timestamp']
for alt in alts:
    lidar_ws_keys.append(' lidar_lidar'+alt+'_Z10_HorizWS')
    lidar_wd_keys.append(' lidar_lidar'+alt+'_WD_alg_03')

In [ ]:
windspeeds=np.asarray(lidar[lidar_ws_keys], dtype='float64')
winddirections=np.asarray(lidar[lidar_wd_keys], dtype='float64')
datetimes=np.asarray(pd.to_datetime(lidar['timestamp']))
heights = [18, 38, 58, 78, 98, 118, 138, 158, 178, 198]

In [ ]:
ta=np.asarray(lidar[" meteo_Ta_avg"], dtype="float64")
pa=np.asarray(lidar[" meteo_Pa_avg"], dtype="float64")
rh=np.asarray(lidar[" meteo_Ua_avg"], dtype="float64")
waterT=np.asarray(lidarWaterT[" ADCP_ADCPtemp"], dtype="float64")

In [ ]:
lidar_xr = xr.Dataset(
    data_vars = dict(
        windspeed=(["t","z"], windspeeds),
        winddirection=(["t","z"], winddirections),
        temp=(["t"], ta),
        pressure=(["t"], pa),
        rel_humidity=(["t"], rh),
        waterT=(["t"], waterT)
    ),
    coords=dict(
        datetime=(["t"], datetimes),
        height=(["z"], heights)
    )
)

In [ ]:
def u_v_from_WS_WD(WS, WD):
    u = WS * np.cos((270 - WD)/180 * np.pi)
    v = WS * np.sin((270 - WD)/180 * np.pi)
    return (u, v)

In [ ]:
(u, v) = u_v_from_WS_WD(lidar_xr['windspeed'], lidar_xr['winddirection'])
lidar_xr['U'] = u
lidar_xr['V'] = v

In [ ]:
lidar_xr.to_netcdf('LiDAR/'+Buoy+'_lidar_full.nc')

In [ ]:
if Buoy == 'E06':
    springtime = lidar_xr.isel(t = slice(29896, 42419))
elif Buoy == 'E05':
    springtime = lidar_xr.isel(t = slice(33552, 46657))
springtime

In [ ]:
springtime.to_netcdf('LiDAR/springtime_'+Buoy+'.nc')

In [ ]:
lidar_xr['winddirection'].min()

### QC & Jets

In [ ]:
def is_data_pt_valid(WS, WD):
    valid = True
    if (WS < 0.0).any():
        valid = False
    elif (WS > 70.0).any():
        valid = False
    elif np.count_nonzero(np.isnan(WS)) > 7:
        valid = False
    return valid

In [ ]:
validity = np.zeros_like(lidar_xr['temp'], dtype='bool')
for i in range(len(validity)):
    validity[i] = is_data_pt_valid(lidar_xr['windspeed'].isel(t=i), lidar_xr['winddirection'].isel(t=i))

In [ ]:
jet_id = findLLJ.findLLJevents_xr_lidar(lidar_xr)

In [ ]:
jet_id = jet_id.rename_dims({'Time': 't'})
jet_id = jet_id.assign(dict(is_valid=(["t"], validity)))
jet_id['datetime'] = lidar_xr['datetime']
jet_id.to_netcdf('LiDAR/full_'+Buoy+'_LLJs.nc')

In [ ]:
#jet_id = xr.load_dataset('LiDAR/full_'+Buoy+'_LLJs.nc')
jet_id = jet_id.assign_coords(t=jet_id['datetime'])
spring_jets = jet_id.sel(t=slice('2020-04-01', '2020-07-01'))
spring_jets.to_netcdf('LiDAR/springtime_'+Buoy+'_LLJs_2020.nc')

In [ ]:
Buoy = 'E06'
jet_id = xr.load_dataset('LiDAR/full_'+Buoy+'_LLJs.nc')
daily_jets = jet_id['is_valid'].rolling(t=24*6).sum()
jet_id['is_valid'].sum() / len(jet_id['is_valid'])

In [ ]:
plt.figure(figsize=(24,3))
plt.plot(jet_id['datetime'], daily_jets)

# SGP Data
Downloaded from https://www.arm.gov/capabilities/instruments/dl for SGP site C1, 2018-06-08 through 2018-06-20. NetCDF file is NOT included in repo.

In [ ]:
lidar = xr.open_dataset('LiDAR/SGP_lidar.nc')
lidar

In [ ]:
data = lidar.sel(site='C1').isel(height = slice(0, 24))
data['windspeed'] = data['wind_speed']
data['datetime'] = data['time']
data = data.rename(dict({'height': 'z', 'time': 't'}))
data['height'] = data['z']

In [ ]:
jet_id = findLLJ.findLLJevents_xr_lidar(data)

In [ ]:
jet_id.to_netcdf('LiDAR/SGP_jet_id.nc')